In [1]:
# IMPORTS
from datetime import datetime
import pandas as pd
import re

### **Dataset Examination & Cleaning**

In [2]:
# LOAD DATASET
raw_dataset = pd.read_csv('congressional-trading-dataset/congress-trading-all.csv')
print(*raw_dataset.columns.to_list(), sep=', ')
raw_dataset = raw_dataset.drop(columns=['TickerType', 'Company', 'Subholding', 'Comments', 'Quiver_Upload_Time', 'Description', 'Status', 'Comments', 'last_modified', 'excess_return'])
raw_dataset.head()

Ticker, TickerType, Company, Traded, Transaction, Trade_Size_USD, Status, Subholding, Description, Name, Filed, Party, District, Chamber, Comments, Quiver_Upload_Time, excess_return, State, last_modified


,Ticker,Traded,Transaction,Trade_Size_USD,Name,Filed,Party,District,Chamber,State
0,NGL,"Monday, March 11, 2024",Sale,"$15,001 - $50,000",Mark Dr Green,2024-03-13,R,TN07,House,Tennessee
1,FCX,"Thursday, February 29, 2024",Sale,"$1,001 - $15,000",Josh Gottheimer,2024-03-07,D,NJ05,House,New Jersey
2,V,"Thursday, February 29, 2024",Purchase,"$1,001 - $15,000",Pete Sessions,2024-03-07,R,TX17,House,Texas
3,APPLE INC. (XNAS:AAPL),"Thursday, February 29, 2024",Purchase,$360.00,Pete Sessions,2024-02-29,R,TX17,House,Texas
4,MSFT,"Thursday, February 29, 2024",Sale,"$100,001 - $250,000",Suzan K. Delbene,2024-03-08,D,WA01,House,Washington


### **Initial Observations**

#### `Ticker` Feature

In [3]:
# FIND/PRINT "WEIRD" TICKERS TO CATEGORIZE/REMOVE, IF APP.
ticker_list = raw_dataset['Ticker'].unique().tolist()
weird_tickers = list(filter(lambda t: len(t)>5, ticker_list))
weird_tickers.sort()
print(f'Of {len(ticker_list)} unique tickers, there are {len(weird_tickers)} "weird" tickers (length > 5 characters)')
print(*weird_tickers, sep=" | ")

Of 3966 unique tickers, there are 123 "weird" tickers (length > 5 characters)
07/01/28 | 07/01/36 | 0QZI.IL | 12/01/37 | 13.WEEK MATURE | 13.WEEK, MATURE | 3.MONTH, MATURE | 3305779 | 36966R5P7 | 3V64.TI | 4.WEEK MATURE | 4.WEEK, MATURE | 40055AY43 | 40056FN84 | 4491235 | 4824778 | 506120KP8 | 5522714 | 6.MONTH | 6.MONTH, MATURE | 605699PU5 | 6320058 | 6897143 | 784532JF1 | 8.WEEK, MATURE | 912796QV4 | 912796QX0 | 912796SH3 | 912796U31 | 912796U49 | 912796WX3 | 912796X87 | 912796XQ7 | 912796YH6 | 912796YJ2 | 912796ZP7 | 912797LL9 | 91282CDR9 | 91282CEG2 | 91282CFG1 | 91282CFN6 | 914476PV2 | AAIC$B | AEX.MU | ALLY.A | APPLE INC. (XNAS:AAPL) | B0T4J94 | B13WZ26 | B1W8P14 | B1WSP14 | B5NC0D0 | B923935 | BAC.PL | BACWSA | BANC$E | BELGIUM | BLM7FC2 | BN7Q3G8 | BOT4J94 | BUNT.RP | CARDONA | CLNS$A | CMLPSX | COF.PP | CORPORAT | CTECBX | CTFS PARTN | CWEN.A | D/B/A ELLIOTT | DUE 04/01/27 | DUE 07/01/25 | DUE 07/01/27 | DUE 08/01/25 | DUE 10/01/203 | DUE 10/01/37 | DUE 12/01/27 | DUE 12/01/29

**Notable Types of "Weird" Tickers**
- **Preferred Stock**: Tickers w/ `$`, `.P`, `.PC`, `.PN`, `.PP`, etc.
  - From [Investopedia](https://www.investopedia.com/articles/01/070401.asp), "an extra letter might be tacked on at the end to identify the type of share or to identify preferred stock."
- **CUSIP**: `^[0-9A-Z]{9}$`, including `91282CDR9`, `506120KP8`, etc.
  - From [Investopedia](https://www.investopedia.com/terms/c/cusipnumber.asp), these are called a  *CUSIP number*, "a series of letters and numbers used to identify certain securities and facilitate trading and settlement," and are used in the United States and Canada. 
- **Funds**: Mutual Funds all have tickers that end with an `X`
- **Option Contracts**: `SPY160219P00180000`, etc.
  - OCC Option symbol (consists of root symbol, exp. date, option type (put/call), strike price)
- **Foreign Stock**: Tickers like `IBM.MX`, `RDSA.AS`, `CWEN.A`, `SYY.SG`, etc.

Can't make confident judgement on other tickers, will likely purge from the dataset

In [4]:
# REPLACE/REMOVE ILL-FORMATTED OR UNIDENTIFIABLE TICKER
def inspect_ticker(ticker: str):
    if len(ticker) <= 5:
        return False
    elif re.match(r"^[0-9A-Z]{9}$", ticker):
        return "Security"
    elif re.match(r"^[A-Z]{1,5}\$[A-Z]{1,3}$", ticker):
        return ticker
    elif re.match(r"^[A-Z]{1,5}\.[A-Z]{1,3}$", ticker):
        return "Foreign_Stock"
    else:
        inside_string = re.search(r"([A-Z]{2,6}):\s*([A-Z]{1,5})", ticker)
        if inside_string:
            return inside_string.group(2)
        else:
            return "REMOVE"

def drop_bad_tickers(dataset: pd.DataFrame):
    df = dataset.copy()
    drop_indices = []
    for index, row in df.iterrows():
        new_ticker = inspect_ticker(row['Ticker'])
        if new_ticker is False:
            continue
        elif new_ticker == "REMOVE":
            print(f"Dropping {row['Ticker']}... (index {index})")
            drop_indices.append(index)
        else:
            df.loc[index, 'Ticker'] = new_ticker
    df = df.drop(index=drop_indices)
    return drop_indices, df

In [5]:
# CLEAN DATASET['TICKERS']
dropped, updated_dataset = drop_bad_tickers(raw_dataset)
print(f"Of the {len(weird_tickers)} 'weird' tickers originally flagged, regex matching narrowed the 'weird' to {len(dropped)} 'bad' tickers, which were dropped!")

Dropping GLAS FUNDS, LP... (index 90)
Dropping HAMILTO... (index 93)
Dropping GLAS FUNDS, LP... (index 137)
Dropping 4.WEEK, MATURE... (index 154)
Dropping 3.MONTH, MATURE... (index 328)
Dropping 6.MONTH, MATURE... (index 486)
Dropping GLAS FUNDS... (index 526)
Dropping GLAS FUNDS, LP... (index 645)
Dropping GLAS FUNDS, LP... (index 1147)
Dropping 13.WEEK, MATURE... (index 1148)
Dropping GLAS FUNDS, LP... (index 1180)
Dropping 4.WEEK, MATURE... (index 1718)
Dropping 3.MONTH, MATURE... (index 1862)
Dropping DUE 10/01/203... (index 1879)
Dropping 8.WEEK, MATURE... (index 2558)
Dropping 4.WEEK, MATURE... (index 2561)
Dropping GLAS FUNDS... (index 2590)
Dropping MATURE... (index 2669)
Dropping BELGIUM... (index 2783)
Dropping STATE OF... (index 2969)
Dropping 4.WEEK, MATURE... (index 2994)
Dropping STATE OF... (index 3195)
Dropping MATURIT... (index 4105)
Dropping MATURIT... (index 4214)
Dropping 13.WEEK, MATURE... (index 4273)
Dropping 4.WEEK MATURE... (index 4518)
Dropping 13.WEEK MATURE

#### `Traded` Feature

**Other attributes** (`Filed`, etc.) with date values adhere to a `YYYY-MM-DD` format, so it follow that this column's values should be converted to match.

In [6]:
# CONVERT LONG DATE STRING INTO MORE COMPACT YYYY-MM-DD FORMAT
def shorten_date_string(date: str):
    dt = datetime.strptime(date, '%A, %B %d, %Y')
    return dt.strftime('%Y-%m-%d')

for index, row in updated_dataset.iterrows():
    updated_dataset.loc[index, 'Traded'] = shorten_date_string(row['Traded'])

#### `Trade_Size_USD` Feature

In [7]:
# REWRITE VALUE INTO APPLICABLE BUCKETS, DISCARD THE REST
trade_size_buckets = ['$1-$1,000',
                      '$1,001 - $15,000',
                      '$15,001 - $50,000',
                      '$50,001 - $100,000',
                      '$100,001 - $250,000',
                      '$250,001 - $500,000',
                      '$500,001 - $1,000,000',
                      '$1,000,001 - $5,000,000',
                      '$5,000,001 - $25,000,000',
                      '$25,000,001 - $50,000,000']

def parse_money(s):
    s = s.strip().replace('$', '').replace(',', '')
    s = re.sub(r'[^\d\.]', '', s)  # Remove trailing . or ,
    return float(s) if s else None

def parse_range(trade_size):
    m = re.match(r'^\s*\$(.+?)\s*-\s*\$(.+?)[\.,]?\s*$', trade_size
    )
    if m:
        low, high = parse_money(m.group(1)), parse_money(m.group(2))
        if low is not None and high is not None:
            return min(low, high), max(low, high)
    m = re.match(r'^\s*\$([\d,]+(?:\.\d{2})?)\s*$', trade_size)
    if m:
        v = parse_money(m.group(1))
        if v is not None:
            return v, v
    return None

def parse_bucket(bucket):
    m = re.match(r'^\$(.+?)[ ]*-[ ]*\$(.+)$', bucket.replace(',', ''))
    if m:
        low = parse_money(m.group(1))
        high = parse_money(m.group(2))
        return bucket, low, high
    return None

bucket_ranges = [parse_bucket(b) for b in trade_size_buckets]

def inspect_trade_size(trade_size: str):
    # Already in canonical bucket
    if trade_size in trade_size_buckets:
        return False
    
    r = parse_range(trade_size)
    if not r:
        return "REMOVE"
    trade_min, trade_max = r

    matching_buckets = []
    for label, bucket_min, bucket_max in bucket_ranges:
        if trade_min >= bucket_min and trade_max <= bucket_max:
            matching_buckets.append(label)
    
    if len(matching_buckets) == 1:
        return matching_buckets[0]
    else:
        return "REMOVE"
    
bucket_ranges = [parse_bucket(b) for b in trade_size_buckets]
def drop_bad_trades(dataset: pd.DataFrame):
    df = dataset.copy()
    drop_indices = []
    for index, row in df.iterrows():
        new_trade_size = inspect_trade_size(row['Trade_Size_USD'])
        if new_trade_size is False:
            continue
        elif new_trade_size == "REMOVE":
            print(f"Dropping {row['Trade_Size_USD']}... (index {index})")
            drop_indices.append(index)
        else:
            df.loc[index, 'Trade_Size_USD'] = new_trade_size
    df = df.drop(index=drop_indices)
    return drop_indices, df

In [8]:
# CLEAN TRADE SIZE ATTRIBUTE
drop_indices, cleaned_df = drop_bad_trades(updated_dataset)

Dropping $1,000.22... (index 15739)
Dropping $.01 - $0.00... (index 15818)
Dropping $6.20 - $100,001... (index 39169)
Dropping $6.20 - $15,001... (index 39174)
Dropping $6.20 - $15,001... (index 39211)
Dropping $1.26 - $50,001... (index 41749)
Dropping $1.26 - $250,001... (index 42000)
Dropping $1.26 - $1,001... (index 42011)
Dropping $250,001-$500,000 - $100,001... (index 42456)
Dropping $50,001-$100,000. - $15,001... (index 42637)


In [10]:
# SAVE "CLEANED" DATASET
cleaned_df.to_csv('prepared_congress_trading_dataset.csv')